### A notebook evaluating a possible implementation for _live Boheh plots in a notebook cell_

In [8]:
from __future__ import print_function
import logging
import socket
import datetime
from collections import OrderedDict
from math import ceil, pi
import six
import ipywidgets as widgets
import numpy as np
from IPython.display import HTML
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.embed import autoload_server
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import row, column, layout, gridplot
from bokeh.models import ColumnDataSource, CustomJS, DatetimeTickFormatter
from bokeh.models import widgets as bkhwidgets
from bokeh.models.glyphs import Rect
from bokeh.models.mappers import LinearColorMapper
from bokeh.models.ranges import Range1d
from bokeh.models.tools import BoxSelectTool, HoverTool, CrosshairTool
from bokeh.models.tools import ResetTool, PanTool, BoxZoomTool, ResizeTool
from bokeh.models.tools import WheelZoomTool, SaveTool
from bokeh.models.widgets import Slider, Button
from bokeh.palettes import Plasma256
from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.resources import INLINE
from bokeh.server.server import Server
from tornado.ioloop import IOLoop
from tools import *

In [9]:
from collections import deque

class EmbeddedBokehServer(object):

    __bkh_app__ = None
    __bkh_srv__ = None
    __srv_url__ = None
    __callbacks__ = deque()

    @staticmethod
    def print_info():
        try:
            print("Bokeh server URL: {}".format(EmbeddedBokehServer.__srv_url__))
            sessions = EmbeddedBokehServer.__bkh_srv__.get_sessions()
            print("Number of opened sessions: {}".format(len(sessions)))
        except:
            pass
        
    @staticmethod
    def __start_server():
        app = Application(FunctionHandler(EmbeddedBokehServer.__entry_point))
        srv = Server(
            {'/': app},
            io_loop=IOLoop.instance(),
            port=0,
            host='*',
            allow_websocket_origin=['*']
        )
        srv.start()
        srv_addr = srv.address if srv.address else socket.gethostbyname(socket.gethostname())
        EmbeddedBokehServer.__bkh_srv__ = srv
        EmbeddedBokehServer.__bkh_app__ = app
        EmbeddedBokehServer.__srv_url__ = 'http://{}:{}'.format(srv_addr, srv.port)
        
    @staticmethod
    def __entry_point(doc):
        try:
            #TODO: should we locked EmbeddedBokehServer.__callbacks__?
            callback = EmbeddedBokehServer.__callbacks__.pop()
            callback(doc)
        except:
            pass

    @staticmethod
    def serve(callback):
        if not EmbeddedBokehServer.__bkh_srv__:
            EmbeddedBokehServer.__start_server() 
        #TODO: should we locked EmbeddedBokehServer.__callbacks__?
        EmbeddedBokehServer.__callbacks__.appendleft(callback)
        script = autoload_server(model=None, url=EmbeddedBokehServer.__srv_url__)
        html_display = HTML(script)
        display(html_display)

In [10]:
def entry_point_c(doc):
    p = figure(plot_width=850, plot_height=200)
    p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
    doc.add_root(p)

In [11]:
def entry_point_s(doc):
    p = figure(plot_width=850, plot_height=200)
    p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="green", alpha=0.5)
    doc.add_root(p)

In [16]:
EmbeddedBokehServer.serve(entry_point_c)

In [17]:
EmbeddedBokehServer.serve(entry_point_s)

In [18]:
EmbeddedBokehServer.print_info()

Bokeh server URL: http://127.0.0.1:57195
Number of opened sessions: 4
